In [1]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import os
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

plt.style.use('fig_publication.mplstyle')
%config InlineBackend.figure_format = 'retina'

# Load fakes (sims) DF and DR4 DF

In [2]:
dr4_df = pd.read_csv("./dataframes/orig_dr4_df.csv.gzip", compression='gzip', index_col=0)

In [3]:
dr4_df

,amplitude_magn_g,anderson_darling_normal_magn_g,beyond_1_std_magn_g,beyond_2_std_magn_g,cusum_magn_g,inter_percentile_range_2_magn_g,inter_percentile_range_10_magn_g,inter_percentile_range_25_magn_g,kurtosis_magn_g,linear_fit_slope_magn_g,...,standard_deviation_magn_r,stetson_K_magn_r,weighted_mean_magn_r,anderson_darling_normal_flux_r,cusum_flux_r,excess_variance_flux_r,kurtosis_flux_r,mean_variance_flux_r,skew_flux_r,stetson_K_flux_r
sid,,,,,,,,,,,,,,,,,,,,,
245101100000933,0.177000,3.696596,0.370000,0.030000,0.158870,0.341499,0.286501,0.158501,-0.787381,-0.000212,...,0.097860,0.851286,15.842624,2.711263,0.099072,0.007498,-0.944294,0.087656,-0.507748,0.844530
245101100009703,0.293500,2.724011,0.290000,0.060000,0.184453,0.523001,0.333000,0.104000,0.201318,-0.001444,...,0.130134,0.869721,15.653200,4.130320,0.156431,0.013800,-1.050300,0.118238,-0.059173,0.909857
245101100010242,0.328500,5.493390,0.190000,0.050000,0.229448,0.559999,0.220499,0.094999,5.286414,-0.001808,...,0.068449,0.672620,16.533829,4.853297,0.187999,0.004273,8.948397,0.067449,2.465220,0.683464
245104200000799,0.329500,2.502895,0.467290,0.000000,0.089224,0.593760,0.490400,0.350000,-1.362500,0.000242,...,0.123850,0.858237,17.937359,1.524028,0.131790,0.011540,-0.980239,0.112776,-0.026306,0.870056
245104200000939,2.293000,12.605539,0.066038,0.056604,0.177457,4.114202,1.156300,0.426999,9.274009,-0.003301,...,0.394907,0.789900,15.854793,1.492581,0.322914,0.081920,0.337605,0.286570,0.109400,0.675794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597113100010642,0.483999,1.274301,0.293578,0.055046,0.187018,0.595680,0.369799,0.157999,1.613005,-0.001550,...,0.107312,0.680863,18.940697,5.417244,0.233112,0.008082,5.852146,0.105139,1.738649,0.670839
1597113100019088,0.323501,1.453954,0.271186,0.076271,0.202899,0.490780,0.252100,0.097000,1.162233,-0.001054,...,0.079516,0.702109,17.359133,3.377403,0.154508,0.005559,5.974216,0.076836,1.682657,0.710270
1597113100031221,0.074500,0.266636,0.307692,0.025641,0.153808,0.093640,0.060800,0.032499,1.197538,-0.000101,...,0.031295,0.753234,13.399863,0.861085,0.155385,0.000762,1.899416,0.029034,0.619902,0.756356


In [4]:
snad_catalog = pd.read_csv('./snad_catalog.csv')
snad_catalog

,Name,R.A.,Dec.,OID,Discovery date (UT),mag,er_up,er_down,ref,er_ref,TNS,Type,Comments
0,SNAD101,247.45543,24.77282,633207400004730,2018-04-08 09:45:49,21.110,0.27,0.36,20.84,0.06,AT 2018lwh,PSN,ZTF18abqkqdm
1,SNAD102,245.05375,28.38220,633216300024691,2018-03-21 11:08:19,21.180,0.28,0.39,20.26,0.07,AT 2018lwi,PSN,ZTF18abdgwos
2,SNAD103,219.83787,47.40378,758205100001118,2018-05-24 07:40:32,21.130,0.28,0.37,21.02,0.07,AT 2018lwj,AGN,NaN
3,SNAD104,218.91620,46.38441,758205400019523,2018-06-24 07:23:27,20.100,0.14,0.16,20.89,0.10,AT 2018lwk,PSN,ZTF18aawqbuc
4,SNAD105,218.34626,49.22553,758209200010983,2018-04-14 07:33:55,21.030,0.26,0.34,21.51,0.10,AT 2018lwl,AGN,NaN
5,SNAD106,219.41935,50.16706,758213400002862,2018-05-04 07:48:50,20.980,0.27,0.36,20.98,0.08,AT 2018lwm,PSN,NaN
6,SNAD107,254.41703,44.29804,762202300014913,2018-05-04 07:39:44,21.400,0.27,0.36,21.82,0.17,AT 2018lwn,PSN,NaN
7,SNAD108,257.84004,48.21127,762209400037712,2018-04-16 10:34:47,21.230,0.28,0.38,20.99,0.06,AT 2018lwo,PSN,ZTF18abauopo
8,SNAD109,191.17808,54.31252,790207100016149,2018-05-04 06:23:51,22.350,0.23,0.29,20.67,0.06,AT 2018lwp,PSN,NaN
9,SNAD110,197.91087,56.96729,790213400013270,2018-05-22 04:56:56,20.640,0.26,0.35,20.22,0.06,AT 2018lwq,PSN,NaN


In [5]:
#snad_catalog_oids = list(snad_catalog.OID) # Doesn't work w/ OIDs from snad catalog (wrong field)

# SNAD 149 - SNAD 159
snad_catalog_oids = list([849102200015533, 
                          679108100003227,
                          790103100000915,
                          792111100012457,
                          643105300009229,
                          719102100006086,
                          718105300007353,
                          755103100026105,
                          684115200018744,
                          634108100006647,
                          792105300007593,
                         ])

oid_df = dr4_df[dr4_df.index.isin(snad_catalog_oids)]
oid_df

,amplitude_magn_g,anderson_darling_normal_magn_g,beyond_1_std_magn_g,beyond_2_std_magn_g,cusum_magn_g,inter_percentile_range_2_magn_g,inter_percentile_range_10_magn_g,inter_percentile_range_25_magn_g,kurtosis_magn_g,linear_fit_slope_magn_g,...,standard_deviation_magn_r,stetson_K_magn_r,weighted_mean_magn_r,anderson_darling_normal_flux_r,cusum_flux_r,excess_variance_flux_r,kurtosis_flux_r,mean_variance_flux_r,skew_flux_r,stetson_K_flux_r
sid,,,,,,,,,,,,,,,,,,,,,
634108100006647,0.788501,4.144565,0.326693,0.031873,0.271581,1.091082,0.741800,0.355999,-0.066709,-0.000598,...,0.208345,0.837546,20.234222,10.895554,0.259617,0.028294,0.145910,0.201797,0.897678,0.792870
643105300009229,0.869500,7.627143,0.341837,0.030612,0.393316,1.256779,0.986601,0.521000,-0.538828,-0.000731,...,0.247104,0.856693,19.835100,16.300173,0.355079,0.055571,0.256303,0.248415,1.129951,0.787710
679108100003227,1.573501,5.977306,0.392308,0.000000,0.458776,2.854298,2.037498,1.579001,-1.377225,0.009025,...,0.872471,0.937693,19.612993,9.534520,0.453928,0.510054,-1.462630,0.719901,0.480915,0.847215
684115200018744,0.934000,7.648891,0.363636,0.005348,0.388977,1.548078,1.236799,0.764000,-0.930156,0.001999,...,0.339097,0.823644,20.288290,3.970340,0.372753,0.081017,-0.034154,0.312301,0.611732,0.805326
718105300007353,0.931500,5.706629,0.428571,0.006211,0.451698,1.559238,1.249599,0.893499,-1.396027,0.002616,...,0.256829,0.896034,19.952436,7.425904,0.424808,0.049328,-0.490650,0.247433,0.733432,0.852638
719102100006086,0.928500,7.237684,0.295455,0.051136,0.341858,1.494640,1.093000,0.517498,-0.367132,0.001753,...,0.313883,0.800681,20.366861,14.123340,0.338642,0.087830,1.675991,0.325879,1.435438,0.741123
755103100026105,1.069500,10.742224,0.212598,0.078740,0.346419,1.710642,1.316799,0.351500,0.729611,0.003222,...,0.347531,0.836197,19.540836,10.939252,0.359712,0.129961,1.375299,0.371685,1.490452,0.753253
790103100000915,1.203501,4.440378,0.561403,0.000000,0.457929,2.253540,1.901300,1.593000,-1.577778,-0.004206,...,0.622673,0.920985,19.885199,12.474462,0.427219,0.367684,-0.578070,0.617471,0.928122,0.824911
792105300007593,0.681000,5.075209,0.331967,0.040984,0.304759,1.165060,0.852600,0.413502,-0.443933,-0.000751,...,0.244994,0.841599,19.749077,12.585350,0.371722,0.051918,0.255619,0.245974,1.126070,0.791670


In [6]:
#fakes_df = pd.concat([fakes_df, oid_df])
#fakes_df

# PCA 

In [7]:
def applypca(dataset_real, dataset_fakes, pca_num):
    X_real = dataset_real
    X_fakes = dataset_fakes
    
    ss = StandardScaler()
    scaled_X_real = ss.fit_transform(X_real)
    pca = PCA(pca_num)
    
    # apply same PCA to both real and fakes data
    pcs_X_real = pca.fit_transform(scaled_X_real)
    pcs_X_fakes = pca.transform(ss.transform(X_fakes))
    
    score = pcs_X_real[:,0:2]
    coeff = np.transpose(pca.components_[0:2, :])
    
    return pca, pcs_X_real, pcs_X_fakes, score, coeff

In [8]:
def pcaplot(score, coeff, datacolor):
    xs = score[:,0]
    ys = score[:,1]
    n = coeff.shape[0]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    plt.scatter(xs * scalex,ys * scaley, c=datacolor, s=6, alpha=0.6) 
    for i in range(n):
        plt.arrow(0, 0, coeff[i,0], coeff[i,1], head_width= .003, color = 'r',alpha = 0.5)
        fakes_df_cols_latex_read = [r.replace('_', '\_') for r in fakes_df.columns.values]
        plt.text(coeff[i,0]* 1.12, coeff[i,1] * 1.12, list(fakes_df_cols_latex_read)[i], 
                     color = 'darkblue', ha = 'center', va = 'center', fontsize = 10)

In [9]:
pca, pcs_X_real, pcs_X_fakes, score, coeff = applypca(dataset_real=dr4_df, 
                                                      dataset_fakes=oid_df, 
                                                      pca_num=25)

# Build DFs from first (15) PCs

In [10]:
pcs_X_oid_df = pd.DataFrame(data=pcs_X_fakes, index=oid_df.index)
pcs_X_oid_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
sid,,,,,,,,,,,,,,,,,,,,,
634108100006647,8.960997,-2.951378,3.757171,0.750713,4.222527,-0.512849,3.088587,-0.215449,-1.025909,-0.113865,...,-0.252645,1.064005,-0.691014,-0.165712,-1.108559,0.489566,-0.386551,0.831991,-0.296872,-0.122569
643105300009229,13.386827,-2.768121,-0.285997,4.149218,7.079689,-0.205226,0.254992,-0.023648,-0.513452,-0.412642,...,-0.781521,0.268433,0.931163,1.093001,-0.172424,-0.291735,1.178079,0.785500,-1.397706,1.737722
679108100003227,47.765515,-4.782633,7.026356,16.748550,-22.399856,-1.299866,-34.883749,-3.152576,5.360719,-3.109600,...,5.023646,-8.870316,8.563166,1.766284,-1.338035,-0.049527,1.008020,6.255714,2.087638,4.338428
684115200018744,18.768984,-4.135084,4.065061,5.628389,-1.531801,-0.545798,-11.102567,-1.979057,0.976708,-0.583960,...,-0.501336,-0.645949,2.066496,0.977431,-0.177786,-1.545719,1.205265,2.014398,-0.720186,1.178658
718105300007353,17.068422,-8.583867,5.966637,7.279004,-2.529017,2.281936,-14.138998,-2.397235,-0.888688,0.022840,...,-2.809952,1.217155,0.162889,3.008569,-1.113579,2.459943,0.133880,2.066677,0.943234,1.123967
719102100006086,15.446059,-1.538959,3.871899,1.730970,-1.375449,-2.782678,-9.280729,-0.478811,1.107357,0.219523,...,-0.122211,-0.224629,-0.051994,0.160857,-0.009375,-0.076590,0.955512,1.012605,-0.483353,0.617230
755103100026105,17.054511,0.109312,4.057258,1.437860,-4.026629,-4.414035,-14.102594,-0.807258,4.880240,-0.051423,...,-0.128818,-1.198324,-0.572571,0.423050,0.923478,2.272888,-0.154347,1.195010,-0.140320,0.330817
790103100000915,33.671394,-5.857337,-0.858255,15.648246,14.730571,2.334192,15.850673,5.325776,-2.232623,-2.989343,...,-4.977277,6.449977,0.500588,1.919820,-3.107903,-1.511997,-2.178798,3.999414,2.913194,2.974920
792105300007593,10.889331,-4.336303,3.339031,2.332783,6.504496,-1.045790,2.076176,0.308325,0.170340,-0.618769,...,-0.448698,1.823852,-0.815718,-0.352466,0.501458,0.951135,-0.433922,1.669031,0.519299,0.102095


In [11]:
pcs_X_real_df = pd.DataFrame(data=pcs_X_real, index=dr4_df.index)
pcs_X_real_df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
sid,,,,,,,,,,,,,,,,,,,,,
245101100000933,1.848374,-4.414712,-5.619388,5.260005,-0.999672,1.886671,2.324748,0.254252,-0.610030,-0.509776,...,2.363753,-2.795466,2.416028,-0.034807,1.239462,0.233966,0.753618,0.509944,1.128944,-1.015804
245101100009703,2.967064,-1.671175,-6.837338,3.595927,4.496302,0.035431,7.659698,0.350599,1.741938,-1.395357,...,1.981555,-2.633424,1.926833,-0.263021,0.650758,0.186640,0.570188,-0.013200,0.117372,0.232071
245101100010242,-0.108842,2.405274,-5.404214,0.148656,7.597006,0.413194,4.990582,0.843978,0.302756,-0.536160,...,0.502832,-1.557730,2.004200,0.348787,0.435873,-1.213873,-0.239160,-0.300573,0.192416,-0.144760
245104200000799,7.365672,-5.579897,-5.080349,6.771783,0.701664,4.046195,2.639496,-1.075730,-2.850413,-0.918207,...,3.194731,-5.855161,3.250182,1.658496,1.870376,3.682299,1.060268,0.077115,0.044291,1.152280
245104200000939,30.154807,10.945092,-8.899698,12.322096,17.629030,-5.978412,18.176991,3.577017,2.804725,-3.233152,...,-4.807445,-11.341741,6.771275,9.332950,4.848983,8.524226,2.171636,-2.899519,-1.822353,-2.429289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597113100010642,3.946898,2.210204,-5.614792,0.057016,10.194056,-0.193863,7.076334,-0.936091,-2.615133,-0.239188,...,1.536704,-3.626290,1.342477,0.646233,1.442203,2.407983,0.877644,0.468168,0.755846,-0.863829
1597113100019088,0.413334,0.540809,-4.052421,-0.078610,4.553594,-0.578823,2.776985,0.468308,-0.761869,-0.078963,...,0.116409,-1.875946,0.258384,0.022575,-0.075510,-0.953803,0.468399,-0.055700,0.828541,-0.108133
1597113100031221,-5.107737,-2.974856,-3.773432,1.666605,-0.358645,0.263058,0.706932,2.276378,0.666740,-0.080428,...,0.428012,-1.025672,0.830843,0.227386,0.396600,0.766338,-0.194758,-0.461759,-0.232381,-0.618600


In [20]:
%%time

# KDTREE
from scipy.spatial import cKDTree

X = np.vstack(pcs_X_oid_df.values)
kdtree = cKDTree(np.vstack(pcs_X_real_df.values))
ktot=20
dist, ind = kdtree.query(X, k=ktot)

# Save PCA kdtree as pickle
#pickle.dump(kdtree,open('./pca_15pcs_orig_kdtree.p','wb'))

CPU times: user 2.57 s, sys: 311 ms, total: 2.88 s
Wall time: 2.9 s


In [21]:
k_neighbor = 1   # change this number to find the NN you want

print(dist[:, k_neighbor])   # drop id; assumes sorted -> see args!
print(ind[:, k_neighbor])    # drop id

[ 3.48431975  5.9826303  17.91006441  3.50652155  4.95741438  4.58650886
  7.24051275  8.52294216  4.40820088  5.73312711  8.23406847]
[831808 477319 827581 302377 468223 467221 500536 466248 831808 723307
 417902]


In [22]:
kdtree_results_df = pd.DataFrame()
kdtree_results_df['sid'] = pcs_X_oid_df.index
kdtree_results_df = kdtree_results_df.set_index('sid')

kdtree_dist_df = kdtree_results_df.copy()

for knn in range(0,ktot):    
    dr4_match_oid, dr4_match_dist = [],[]
    for ix, dis in zip(ind[:, knn], dist[:, knn]):
        dr4_match_oid.append(dr4_df.iloc[ix].name)
        dr4_match_dist.append(dis)
    kdtree_results_df[f'knn={knn+1}_match_OID'] = dr4_match_oid
    kdtree_dist_df[f'knn={knn+1}_dist'] = dr4_match_dist

In [23]:
#kdtree_results_df.to_csv("./dataframes/pca_15pcs_orig_kdtree_results_df.csv.gzip", compression='gzip')
kdtree_results_df

,knn=1_match_OID,knn=2_match_OID,knn=3_match_OID,knn=4_match_OID,knn=5_match_OID,knn=6_match_OID,knn=7_match_OID,knn=8_match_OID,knn=9_match_OID,knn=10_match_OID,knn=11_match_OID,knn=12_match_OID,knn=13_match_OID,knn=14_match_OID,knn=15_match_OID,knn=16_match_OID,knn=17_match_OID,knn=18_match_OID,knn=19_match_OID,knn=20_match_OID
sid,,,,,,,,,,,,,,,,,,,,
634108100006647,634108100006647,824104400006476,722104400009937,676112300007587,797112400009341,763115300012862,792105300007593,757105300016091,760115300009766,682112400005030,682111400016634,676106400004189,717110300001159,634113400006044,790111400014844,765114300007418,845115200011818,789115400000127,760109100005396,760104300028234
643105300009229,643105300009229,718114300002492,821101400005691,643103200011226,789113400005542,755205100004547,849104200009087,695115400008617,813106300002831,695111100020844,864110300004380,726105400007377,760101200001435,828103200013194,865106100001443,865107100006853,754113100005518,792105300007593,735113300008074,692101200004919
679108100003227,679108100003227,823103400007278,827111100013593,796115400009292,720109400001767,723103100016148,680109100003419,789109400001098,684115400029891,838116300012131,682109200014170,759102100014462,839103300014407,788112400001260,679102200005527,757115100002190,698103300001587,824113100002714,826107200007778,762104400009780
684115200018744,684115200018744,676104100007808,587106400017817,844110200002092,797205400001217,633105400014971,822101200002926,676102400000856,676102100009347,794105300002705,676102300004634,843110100000083,716107200000408,797205100006765,635104300022736,717105400003201,843111200004238,762114100000051,756102100017486,718101100007827
718105300007353,718105300007353,716109300004859,672115200000744,844107100000859,754102300011869,821115300012972,672114100000018,790105300000737,716103200003136,756104100002162,716114100002228,754107200004073,760112400004567,790111400004261,843111200004238,819111400003349,671114100001115,673105400003473,756104400001989,718116100005390
719102100006086,719102100006086,716105400011719,635106200001974,797205100006765,690113100027544,796108200017608,795105100010734,724105200000301,757106100007885,725116100004039,681106400000714,821110200001558,717105300005220,634110100012027,719105100001123,761101200003544,713113200005451,676102100009347,587101400036725,717107300004543
755103100026105,755103100026105,723104100019341,723116300010979,761108300003655,587105300022592,594105100001770,683106100033432,633116300007188,635111400000448,719110300000085,782113300000427,828114400001729,586214200014641,848110100003350,759104400001258,719102100006086,752113100002574,635105200015692,764108300010755,821110200001558
790103100000915,790103100000915,716103100003533,681115400012617,791111200004614,717105400005221,718111100005488,583101400001339,625107100003750,720101300011092,838114400002538,790112100001528,648109300002609,817110200003664,794114200001172,713115300001120,677114100008407,858115100007217,765110200019091,784108400001073,857105300003159
792105300007593,792105300007593,824104400006476,676106400004189,634113400006044,819109100000205,634108100006647,722104400009937,690104100004062,844114100000424,762103100001812,725112400013567,760101200001435,682111400016634,791103100006401,795111300009817,755109400000926,862109200003060,797112400009341,798109300000355,763104100009392


In [24]:
#kdtree_dist_df.to_csv("./dataframes/pca_15pcs_orig_kdtree_dist_df.csv.gzip", compression='gzip')
kdtree_dist_df

,knn=1_dist,knn=2_dist,knn=3_dist,knn=4_dist,knn=5_dist,knn=6_dist,knn=7_dist,knn=8_dist,knn=9_dist,knn=10_dist,knn=11_dist,knn=12_dist,knn=13_dist,knn=14_dist,knn=15_dist,knn=16_dist,knn=17_dist,knn=18_dist,knn=19_dist,knn=20_dist
sid,,,,,,,,,,,,,,,,,,,,
634108100006647,0.020750,3.484320,4.098468,4.305224,4.709590,4.713542,4.863651,5.032156,5.050542,5.095392,5.175076,5.254167,5.279860,5.298526,5.309336,5.322214,5.329463,5.332369,5.339018,5.352935
643105300009229,0.016819,5.982630,6.377466,6.935289,7.017472,7.033170,7.201358,7.245670,7.570983,7.812712,7.858935,7.887914,7.892869,7.943911,7.960337,7.973756,7.995278,8.014632,8.037126,8.041810
679108100003227,0.138268,17.910064,18.685522,19.119062,19.421171,20.258559,24.119323,24.231783,24.319857,25.176354,25.280517,25.323349,26.409424,26.603945,26.744816,26.934195,27.232014,27.294039,27.452007,27.659214
684115200018744,0.014831,3.506522,4.649659,5.477530,5.916352,6.104275,6.565814,6.642994,6.712888,6.722315,6.830450,6.854888,6.859993,6.878753,6.907947,6.924602,6.971264,6.995359,7.185677,7.205505
718105300007353,0.012806,4.957414,5.273190,5.328154,5.401788,5.548964,5.884269,5.887310,5.897150,5.959495,6.229592,6.405253,6.506642,6.517223,6.602311,6.759245,6.944601,6.980019,7.072784,7.195876
719102100006086,0.011686,4.586509,4.603029,4.746278,4.798633,4.814539,4.906406,5.677739,5.710154,5.752577,5.799650,5.829299,5.885900,5.930602,5.955162,6.031288,6.113400,6.187211,6.187811,6.234975
755103100026105,0.029966,7.240513,7.540337,7.623212,7.688133,7.814426,7.843699,8.046749,8.053631,8.328111,8.373332,8.449829,8.453490,8.520301,8.604922,8.608870,8.714474,8.728026,8.729230,8.738293
790103100000915,0.058048,8.522942,9.135009,11.377286,12.012652,12.237963,12.800409,13.726736,13.833048,14.358002,14.579238,15.095346,15.196503,15.539615,15.772678,15.827688,16.035264,16.477267,16.739785,16.759856
792105300007593,0.017519,4.408201,4.593200,4.725680,4.791136,4.860186,5.120983,5.331404,5.491070,5.523502,5.605080,5.605949,5.663509,5.839186,5.948420,5.951016,5.972783,5.999220,6.047586,6.050910


In [25]:
# Total real objects matched
print("Total real objects matched:", len(np.hstack(kdtree_results_df.values)))

# Of above, total UNIQUE real objects matched
print("Total real UNIQUE objects matched:", len(np.unique(np.hstack(kdtree_results_df.values))))
print(np.unique(np.hstack(kdtree_results_df.values)))

Total real objects matched: 220
Total real UNIQUE objects matched: 205
[545114100008154 578101400000953 583101400001339 586116100012463
 586214200014641 587101400036725 587105300022592 587106400017817
 594105100001770 603109400005189 625101400029189 625107100003750
 633105400014971 633113400015478 633116300007188 634108100006647
 634110100012027 634113400006044 635104300022736 635105200015692
 635106200001974 635111400000448 643103200011226 643105300009229
 648109300002609 667113400000692 671114100001115 672114100000018
 672115200000744 673105400003473 676102100009347 676102300004634
 676102400000856 676102400008406 676104100007808 676106400004189
 676112300007587 677114100008407 679102200005527 679108100003227
 680109100003419 681106400000714 681115400012617 682109200014170
 682111400016634 682112400005030 683103300000416 683106100033432
 684115200018744 684115400029891 690104100004062 690113100027544
 692101200004919 695108100008107 695111100020844 695111300015363
 695111300035793 69

# Visually inspect matched OIDs

In [26]:
from IPython.display import display_markdown

print([display_markdown(f'https://ztf.snad.space/view/{oid}', raw=True) for oid in \
         np.unique(np.hstack(kdtree_results_df.values))])

https://ztf.snad.space/view/545114100008154

https://ztf.snad.space/view/578101400000953

https://ztf.snad.space/view/583101400001339

https://ztf.snad.space/view/586116100012463

https://ztf.snad.space/view/586214200014641

...

https://ztf.snad.space/view/865106100001443

https://ztf.snad.space/view/865107100006853

https://ztf.snad.space/view/865110400004308

https://ztf.snad.space/view/865116100005870

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [27]:
from IPython.display import display_markdown

for knn in range(0,ktot):
    print(f"KNN {knn+1} \n")
    _ = [display_markdown(f'https://ztf.snad.space/view/{oid}', raw=True) for oid in \
         kdtree_results_df[f'knn={knn+1}_match_OID']]

KNN 1 



https://ztf.snad.space/view/634108100006647

https://ztf.snad.space/view/643105300009229

https://ztf.snad.space/view/679108100003227

https://ztf.snad.space/view/684115200018744

...

https://ztf.snad.space/view/821110200001558

https://ztf.snad.space/view/857105300003159

https://ztf.snad.space/view/763104100009392

https://ztf.snad.space/view/758111400001358

https://ztf.snad.space/view/683103300000416

https://ztf.snad.space/view/718114300002492  SN Ia?/AGN. Not in Alert Stream
https://ztf.snad.space/view/676104100007808  SN Ia?/AGN. Not in Alert Stream
https://ztf.snad.space/view/716105400011719  SN Ia?/AGN. Not in Alert Stream
https://ztf.snad.space/view/790114200012483  SN Ia?/AGN. Not in Alert Stream
https://ztf.snad.space/view/723104100019341  SN. Not in Alert Stream.
https://ztf.snad.space/view/722104400009937  SN Ia. Not in Alert Stream.
https://ztf.snad.space/view/821101400005691  SN Ia?/AGN. Not in Alert Stream
https://ztf.snad.space/view/587106400017817  SN.
https://ztf.snad.space/view/635106200001974  fall of SN/ QSO
https://ztf.snad.space/view/825114400003486  SN.
https://ztf.snad.space/view/796115400009292  SN.
https://ztf.snad.space/view/797205100006765  SN. Not in Alert Stream
https://ztf.snad.space/view/634113400006044  SN. Not in Alert Stream
https://ztf.snad.space/view/789113400005542  SN.
https://ztf.snad.space/view/797205400001217  Maybe SN.
https://ztf.snad.space/view/587105300022592  SN.
https://ztf.snad.space/view/717105400005221  SN, uncataloged (seen by Masha previously)
https://ztf.snad.space/view/819109100000205  SN
https://ztf.snad.space/view/765116400008586  SN Ia, maybe
https://ztf.snad.space/view/633105400014971  SN/AGN
https://ztf.snad.space/view/796108200017608  SN.
https://ztf.snad.space/view/865110400004308  SN.
https://ztf.snad.space/view/822101200002926  SN.
https://ztf.snad.space/view/762105400002163  SN.
https://ztf.snad.space/view/757105300016091  SN.
https://ztf.snad.space/view/676102400000856  SN.
https://ztf.snad.space/view/760115300009766  SN. Not in alert stream
https://ztf.snad.space/view/757106100007885  SN.
https://ztf.snad.space/view/676102100009347  SN.
https://ztf.snad.space/view/635111400000448  Bright SN.
https://ztf.snad.space/view/794105300002705  SN.
https://ztf.snad.space/view/682111400016634  SN.
https://ztf.snad.space/view/681106400000714  SN.
https://ztf.snad.space/view/821110200001558  SN.
https://ztf.snad.space/view/586214200014641  SN.
https://ztf.snad.space/view/634113400006044  SN/AGN
https://ztf.snad.space/view/828103200013194  SN/TDE?
https://ztf.snad.space/view/797205100006765  SN.
https://ztf.snad.space/view/634110100012027  SN.
https://ztf.snad.space/view/765114300007418  SN/TDE? Not in Alert stream.
https://ztf.snad.space/view/761101200003544  SN.
https://ztf.snad.space/view/761101200003544  SN. Not in Alert stream
https://ztf.snad.space/view/754113100005518  SN/AGN
https://ztf.snad.space/view/713113200005451  SN. Not in Alert stream.
https://ztf.snad.space/view/762114100000051  SN
https://ztf.snad.space/view/676102100009347  SN.
https://ztf.snad.space/view/797112400009341  SN/AGN
https://ztf.snad.space/view/726105200013539  SN/AGN. Not in Alert stream.
https://ztf.snad.space/view/760109100005396  SN.
https://ztf.snad.space/view/692101200004919  SN.
https://ztf.snad.space/view/717107300004543  SN. Not in Alert stream
https://ztf.snad.space/view/821110200001558  SN.
https://ztf.snad.space/view/683103300000416  SN. Not in Alert stream